### Acme Manufacturing Company as a contract to deliver 100, 250, 190, 140, 200 and 110 home windows over the next 6 months. Production cost (labor, material and utility) per window varies by period and is estimated to be \\$50, \\$45, \\$55, \\$48, \\$52 and \\$50 over the next 6 months. To take advantage of the fluctuations in manufacturing cost, Acme can produce more windows than needed in a given month and hold the extra units for delivery in later months. This will incur a storage cost at the rate of \\$8 per window per month, assessed on end-of-month inventory. Develop a linear program to determine the optimum production schedule

### What is the objective function of this problem? How about Decision variables and Constraints? Try and write out the mathematical formulation of the problem

### Solve using PuLP

In [1]:
from pulp import *

#### Create lists or dataframes with data: production cost per month, demand per month, inventory cost

In [2]:
# Constants

month1 = {
    "TARGET": 100,
    "COST": 50
}

month2 = {
    "TARGET": 250,
    "COST": 45
}

month3 = {
    "TARGET": 190,
    "COST": 55
}

month4 = {
    "TARGET": 140,
    "COST": 48
}

month5 = {
    "TARGET": 200,
    "COST": 52
}

month6 = {
    "TARGET": 110,
    "COST": 50
}

storage_cost = 8

total_windows = month1["TARGET"] + month2["TARGET"] + month3["TARGET"] + \
                month4["TARGET"] + month5["TARGET"] + month6["TARGET"]


#### create problem object in pulp 

In [3]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Window Production Cost", LpMinimize)

In [4]:
# Dictionaries created to contain the referenced Variables
p1=LpVariable("Production-Month1", lowBound=100,cat="integer")
p2=LpVariable("Production-Month2", lowBound=0,cat="integer")
p3=LpVariable("Production-Month3", lowBound=0,cat="integer")
p4=LpVariable("Production-Month4", lowBound=0,cat="integer")
p5=LpVariable("Production-Month5", lowBound=0,cat="integer")
p6=LpVariable("Production-Month6", lowBound=0,cat="integer")

# Each of these variables represent the amount of extra production in a given month and delivered in a subsequent month
# e1e2 -- Extra production in month 1 and delivered in month 3
e1d2=LpVariable("ExtraMonth1-Delivered2", lowBound=0, cat="integer")
e1d3=LpVariable("ExtraMonth1-Delivered3", lowBound=0, cat="integer")
e1d4=LpVariable("ExtraMonth1-Delivered4", lowBound=0, cat="integer")
e1d5=LpVariable("ExtraMonth1-Delivered5", lowBound=0, cat="integer")
e1d6=LpVariable("ExtraMonth1-Delivered6", lowBound=0, cat="integer")

e2d3=LpVariable("ExtraMonth2-Delivered3", lowBound=0, cat="integer")
e2d4=LpVariable("ExtraMonth2-Delivered4", lowBound=0, cat="integer")
e2d5=LpVariable("ExtraMonth2-Delivered5", lowBound=0, cat="integer")
e2d6=LpVariable("ExtraMonth2-Delivered6", lowBound=0, cat="integer")

e3d4=LpVariable("ExtraMonth3-Delivered4", lowBound=0, cat="integer")
e3d5=LpVariable("ExtraMonth3-Delivered5", lowBound=0, cat="integer")
e3d6=LpVariable("ExtraMonth3-Delivered6", lowBound=0, cat="integer")

e4d5=LpVariable("ExtraMonth4-Delivered5", lowBound=0, cat="integer")
e4d6=LpVariable("ExtraMonth4-Delivered6", lowBound=0, cat="integer")

e5d6=LpVariable("ExtraMonth5-Delivered6", lowBound=0, cat="integer")

In [5]:
# The objective function is added to 'prob' first
# Cost = SUM of monthly production plus storage costs from one month to the next
# Month 1 has no storage costs as it is the first month
# Month 2 has its production cost plus storage cost if any excess month 1 production is held for month 2
prob += month1["COST"] * (p1 + e1d2 + e1d3 + e1d4 + e1d5 + e1d6) + \
        month2["COST"] * (p2 + e2d3 + e2d4 + e2d5 + e2d6)  + storage_cost * e1d2 + \
        month3["COST"] * (p3 + e3d4 + e3d5 + e3d6)         + storage_cost * (2*e1d3 + e2d3) + \
        month4["COST"] * (p4 + e4d5 + e4d6)                + storage_cost * (3*e1d4 + 2*e2d4 + e3d4) + \
        month5["COST"] * (p5 + e5d6)                       + storage_cost * (4*e1d5 + 3*e2d5 + 2*e3d5 + e4d5) + \
        month6["COST"] * (p6)                              + storage_cost * (5*e1d6 + 4*e2d6 + 3*e3d6 + 2*e4d6 + e5d6)

In [6]:
# add constraints
# Month 1 must deliver month 1's target.
prob += p1 == month1["TARGET"]

# Month 1 production must be at least 100 or more to meet month1 target
prob += p1 + e1d2 + e1d3 + e1d4 + e1d5 + e1d6 >= month1["TARGET"]

# Month 2 production must meet or exceed its monthly target. This may consist of excess month 1 production to
# be delivered in month 2, or excess production to be delivered thru months 3 to 6.
prob += p2 + e2d3 + e2d4 + e2d5 + e2d6 + e1d2 >= month2["TARGET"]

# Month 3
prob += p3 + e3d4 + e3d5 + e3d6 + e1d3 + e2d3 >= month3["TARGET"]

# Month 4
prob += p4 + e4d5 + e4d6 + e1d4 + e2d4 + e3d4 >= month4["TARGET"]

# Month 5
prob += p5 + e5d6 + e1d5 + e2d5 + e3d5 + e4d5 >= month5["TARGET"]

# Month 6
prob += p6 + e1d6 + e2d6 + e3d6 + e4d6 + e5d6 == month6["TARGET"]

# total delivered
prob += p1 + e1d2 + e1d3 + e1d4 + e1d5 + e1d6 + \
        p2 + e2d3 + e2d4 + e2d5 + e2d6 + \
        p3 + e3d4 + e3d5 + e3d6 + \
        p4 + e4d5 + e4d6 + \
        p5 + e5d6 + \
        p6 == total_windows

In [7]:
# The problem data is written to an .lp file
prob.writeLP("KudrykSolution.lp")

In [8]:
# The problem is solved using PuLP's choice of Solver
prob.solve()
print ("Solution time: " + str(prob.solutionTime))

# The status of the solution is printed to the screen
print ("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value

for v in prob.variables():
    print (v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen

prob.objective.value()

Solution time: 0.007621999999999907
Status: Optimal
ExtraMonth1_Delivered2 = 0.0
ExtraMonth1_Delivered3 = 0.0
ExtraMonth1_Delivered4 = 0.0
ExtraMonth1_Delivered5 = 0.0
ExtraMonth1_Delivered6 = 0.0
ExtraMonth2_Delivered3 = 190.0
ExtraMonth2_Delivered4 = 0.0
ExtraMonth2_Delivered5 = 0.0
ExtraMonth2_Delivered6 = 0.0
ExtraMonth3_Delivered4 = 0.0
ExtraMonth3_Delivered5 = 0.0
ExtraMonth3_Delivered6 = 0.0
ExtraMonth4_Delivered5 = 0.0
ExtraMonth4_Delivered6 = 0.0
ExtraMonth5_Delivered6 = 0.0
Production_Month1 = 100.0
Production_Month2 = 250.0
Production_Month3 = 0.0
Production_Month4 = 140.0
Production_Month5 = 200.0
Production_Month6 = 110.0


48940.0